In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from matplotlib import pyplot as plt

from vrAnalysis2.database import get_database
from vrAnalysis2.syd.reliability_viewers import (
    ReliabilityTrajectory,
    FractionActiveTrajectory,
    ReliabilitySingleSession,
    ReliabilityPlasticity,
    ReliabilityStability,
    ReliabilityToSpkmap,
    ReliabilityQuantileSummary,
    ReliabilityQuantileFigureMaker,
    FractionActiveQuantileFigureMaker,
    SingleCellReliabilityFigureMaker,
)
from vrAnalysis2.syd.reliability_continuity import ReliabilityStabilitySummary, ReliabilityMasterSummary
from vrAnalysis2.syd.placecell_reliability import (
    TrackedSpkmapViewer,
    TrackingStatsFigureMaker,
    ConsistentReliabilityFigureMaker,
    ChangingPlaceFieldFigureMaker,
)

sessiondb = get_database('vrSessions')
mousedb = get_database('vrMice')
tracked_mice = mousedb.get_table(tracked=True)["mouseName"].unique()

plt.close('all')

In [8]:
show_viewer = False
if show_viewer:
    rv = ReliabilityQuantileSummary(tracked_mice).deploy(env="notebook")

In [18]:
show_viewer = False
if show_viewer:
    rv = ReliabilityQuantileFigureMaker(tracked_mice, try_cache=True).deploy(env="notebook")

In [6]:
show_viewer = False
if show_viewer:
    rv = FractionActiveQuantileFigureMaker(tracked_mice, try_cache=True).deploy(env="notebook")

In [5]:
show_viewer = False
if show_viewer:
    rv = ReliabilityTrajectory(tracked_mice).deploy(env="notebook")

In [7]:
show_viewer = False
if show_viewer:
    rv = FractionActiveTrajectory(tracked_mice).deploy(env="notebook")

In [10]:
show_viewer = False
if show_viewer:
    rv = TrackedSpkmapViewer(tracked_mice).deploy()

In [14]:
show_viewer = False
if show_viewer:
    # For settings in the saved example, I used ctl: 37, red: 4
    rv = TrackingStatsFigureMaker(tracked_mice, try_cache=True).deploy()

In [7]:
show_viewer = False
if show_viewer:
    # Saved Examples: ctl=5, red=7, reference=1
    rv = ConsistentReliabilityFigureMaker(tracked_mice).deploy()

In [4]:
show_viewer = False
if show_viewer:
    # Example: ctl=15, red=1
    rv = ChangingPlaceFieldFigureMaker(tracked_mice).deploy()

In [17]:
show_viewer = False
if show_viewer:
    rv = ReliabilitySingleSession(tracked_mice).deploy(env="notebook")

In [25]:
show_viewer = False
if show_viewer:
    viewer = ReliabilityPlasticity(tracked_mice).deploy(env="notebook")

In [20]:
show_viewer = False
if show_viewer:
    rv = ReliabilityStability(tracked_mice).deploy(env="notebook")

In [29]:
show_viewer = False
if show_viewer:
    rv = ReliabilityStabilitySummary(tracked_mice, use_cache=False).deploy(env="notebook")

In [6]:
show_viewer = False
if show_viewer:
    rv = ReliabilityToSpkmap(tracked_mice).deploy(env="notebook")

In [8]:
show_viewer = False
if show_viewer:
    rv = SingleCellReliabilityFigureMaker(tracked_mice).deploy()

In [19]:
show_viewer = False
if show_viewer:
    rv = ReliabilityMasterSummary(tracked_mice).deploy(env="notebook")